In [1]:
import configparser
import psycopg2
from sql_queries import copy_table_queries, insert_table_queries

In [2]:
def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()


def insert_tables(cur, conn):
    for query in insert_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

In [5]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()
    
#load_staging_tables(cur, conn)
insert_tables(cur, conn)

conn.close()


        INSERT INTO users (user_id, first_name, last_name, gender, level)
        SELECT 
            DISTINCT(userid) as user_id,
            firstname as first_name,
            lastname as last_name,
            gender,
            level
        FROM staging_events;



InternalError_: Cannot insert a NULL value into column user_id
DETAIL:  
  -----------------------------------------------
  error:  Cannot insert a NULL value into column user_id
  code:      8007
  context:   query execution
  query:     3435
  location:  column:1
  process:   query1_126_3435 [pid=14130]
  -----------------------------------------------

